# Clustering Assignment NLP - Fall 2016
## Clustering Pride and Prejudice - does chapter clustering follow Darcy's proposals?
### michelle.carney@berkeley.edu 20161118
Initial clustering code is taken from brandonrose.org/clustering and past assignments from this course

In [1]:
##All of the imports......
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
import math
import pprint
import matplotlib
from nltk import word_tokenize
sent_tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
from collections import Counter
from __future__ import division
from nltk.collocations import *
import string, random
from nltk.corpus import brown
from nltk.collocations import *
from string import punctuation
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn


First let's get our text to look like Brandon Roses....

In [26]:
pptext = open("prideandprejudice.txt","r")
rawbook = pptext.read()

indices = [m.start() for m in re.finditer('Chapter', rawbook)]
# print(indices)

d = 'Chapter'
start_index = 0
chapters = []

# print(rawbook[indices[0]])
for index in indices:
    chapters.append(rawbook[start_index:index])
    start_index = index
# print(chapters)

titles = [] #titles is actually the chapter 'titles'
synopses = [] #synopses is actually what is in the chapter
for chapter in chapters:
    titles.append(chapter[0:10].strip())
    synopses.append(chapter[10::].strip())

titles.pop(0)
synopses.pop(0)
# print(titles)
# print(synopses)


''

In [28]:
print(titles[:60])

['Chapter 1', 'Chapter 2', 'Chapter 3', 'Chapter 4', 'Chapter 5', 'Chapter 6', 'Chapter 7', 'Chapter 8', 'Chapter 9', 'Chapter 10', 'Chapter 11', 'Chapter 12', 'Chapter 13', 'Chapter 14', 'Chapter 15', 'Chapter 16', 'Chapter 17', 'Chapter 18', 'Chapter 19', 'Chapter 20', 'Chapter 21', 'Chapter 22', 'Chapter 23', 'Chapter 24', 'Chapter 25', 'Chapter 26', 'Chapter 27', 'Chapter 28', 'Chapter 29', 'Chapter 30', 'Chapter 31', 'Chapter 32', 'Chapter 33', 'Chapter 34', 'Chapter 35', 'Chapter 36', 'Chapter 37', 'Chapter 38', 'Chapter 39', 'Chapter 40', 'Chapter 41', 'Chapter 42', 'Chapter 43', 'Chapter 44', 'Chapter 45', 'Chapter 46', 'Chapter 47', 'Chapter 48', 'Chapter 49', 'Chapter 50', 'Chapter 51', 'Chapter 52', 'Chapter 53', 'Chapter 54', 'Chapter 55', 'Chapter 56', 'Chapter 57', 'Chapter 58', 'Chapter 59', 'Chapter 60']


In [29]:
print(synopses[0][:200]) #first 200 characters in first synopses (for 'Chapter 1')

It is a truth universally acknowledged, that a single man in possession
of a good fortune, must be in want of a wife.

However little known the feelings or views of such a man may be on his
first ente


In [30]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [31]:
#here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [32]:
#not super pythonic, no, not at all.
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in synopses:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [33]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

print(vocab_frame.head())

there are 121205 items in vocab_frame
               words
it                it
is                is
a                  a
truth          truth
univers  universally


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(synopses) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 4.11 s, sys: 42.3 ms, total: 4.15 s
Wall time: 4.29 s
(60, 746)


In [35]:
terms = tfidf_vectorizer.get_feature_names()

In [36]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)
print(dist)

[[ -2.22044605e-16   5.74433351e-01   6.46789546e-01 ...,   8.02659515e-01
    6.45181669e-01   7.77070883e-01]
 [  5.74433351e-01   0.00000000e+00   5.83218362e-01 ...,   7.08211511e-01
    6.27087153e-01   8.23754811e-01]
 [  6.46789546e-01   5.83218362e-01   0.00000000e+00 ...,   7.56108860e-01
    6.52415916e-01   7.73313123e-01]
 ..., 
 [  8.02659515e-01   7.08211511e-01   7.56108860e-01 ...,  -2.22044605e-16
    6.00938213e-01   5.98233369e-01]
 [  6.45181669e-01   6.27087153e-01   6.52415916e-01 ...,   6.00938213e-01
   -2.22044605e-16   6.48156471e-01]
 [  7.77070883e-01   8.23754811e-01   7.73313123e-01 ...,   5.98233369e-01
    6.48156471e-01  -4.44089210e-16]]


# Part 1: Experimentation

### My hypothesis is that chapters centered around the main plot point of proposal will be grouped together. In particular, it will be the 3 chunks of: Pre-Darcy proposal 1 (start - ch 34), post-Darcy proposal 1 and pre-Darcy proposal 2 (ch34 - 58), and post-darcy proposal 2 (ch58 - end).
----------------------
*Instructions:
(a) Experiment with either k-means clustering or LDA on your adopted document collection to try to find topics in the collection.   Be sure to try a few different values of k.  (If you want to use some other variant of clustering, that is fine.)
(b) Show your output in some easy-to-digest form.
(c) Discuss how well it did or did not work.*

## Discuss how well it worked:

My hypothesis is that chapters centered around the main plot point of proposal will be grouped together. In particular, it will be the 3 chunks of: Pre-Darcy proposal 1 (start - ch 34), post-Darcy proposal 1 and pre-Darcy proposal 2 (ch34 - 58), and post-darcy proposal 2 (ch58 - end).

If my hypothesis was correct, I would see the clusters for chapters 1-34, 34-58, and 58-60 when I do k=3. Instead, with k=3, I find there is a cluster that consists of chapters 1-18, a few chapters in the middle, and quite a few chapters at the end. When we look at this cluster in particular, we see that some of the words that determine this are ‘dance’ and ‘netherfield’ - which would make sense to the plot, initially the book takes place at a dance at netherfield, and in the end the plot also is around nether field and a dance. This lends more insight into the actual plot of the text - there is a circular motif about being ‘close to home’ or ‘home is where the heart is’ where Elizabeth and Darcy fall in love for the first time and get engaged.

The second cluster in my hypothesis (k=3) is chapters around writing letters back and forth between family members, and my last cluster is chapters focused around family relationships and societal structures. These can also be seen as ‘motif’ or ’themed’ chapters: the second cluster motif being maintaining relationships over distance, and the last cluster being around the importance of family structure.

While this does not match my hypothesis that chapters will cluster relative to the plot (mainly Darcy’s proposals), it is interesting to see how the motifs cluster the chapters. I expanded the number of words to 10 so I could see other words besides titles (mr., mrs.) and names, and these also support my claim of clustering around motifs. I also wanted to keep in titles (like mr. and mrs.) because Elizabeth’s parents are referred to as Mr. Bingley and Mrs. Bingley respectively, but I found they were deciding words in the clustering. 

I also tried to increase the cluster size, and I included my output from k=4 and k=10 as well. I found that k=4 did not support my “home is where the heart is” theme/motif claim - in fact, it did not cluster the beginning chapters and end chapters together like we saw in k=3. The k=4 clusters ended up clustering the chapters by people mentioned in the chapters. The k=10 clusters ended up clustering the chapters more by places or people in those chapters. I believe this is why the k=3 clustering is the most interesting of the clusters because it is more abstracted away from the people and places and kind of gets at overall theme.

# K-Means Clustering
## k = 3 (my hypothesis)

In [37]:
from sklearn.cluster import KMeans

num_clusters = 3

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 513 ms, sys: 5.57 ms, total: 518 ms
Wall time: 537 ms


In [38]:
from sklearn.externals import joblib

#uncomment the below to save your model 
#since I've already run my model I am loading from the pickle

joblib.dump(km,  'doc_cluster.pkl')

km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [39]:
ppbook = {'title': titles, 'synopsis': synopses, 'cluster': clusters}

frame = pd.DataFrame(ppbook, index = [clusters] , columns = ['title', 'cluster'])
frame['cluster'].value_counts() #number of chapters per cluster

1    22
2    21
0    17
Name: cluster, dtype: int64

In [40]:
from __future__ import print_function

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :10]: #replace 10 with n words per cluster
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print() #add whitespace
    print() #add whitespace
    
    print("Cluster %d titles:" % i, end='')
    for title in frame.ix[i]['title'].values.tolist():
        print(' %s,' % title, end='')
    print() #add whitespace
    print() #add whitespace
    
print()
print()

Top terms per cluster:

Cluster 0 words: b'collins', b'mr.', b'lady', b'catherine', b'charlotte', b'ladyship', b'rosings', b'daughters', b'cousin', b'sir',

Cluster 0 titles: Chapter 13, Chapter 14, Chapter 15, Chapter 16, Chapter 19, Chapter 20, Chapter 22, Chapter 23, Chapter 28, Chapter 29, Chapter 30, Chapter 31, Chapter 32, Chapter 37, Chapter 38, Chapter 56, Chapter 57,

Cluster 1 words: b'bingley', b'miss', b'mr.', b'mr.', b'dance', b'mrs.', b'dear', b'mother', b'netherfield', b'hurst',

Cluster 1 titles: Chapter 1, Chapter 2, Chapter 3, Chapter 4, Chapter 5, Chapter 6, Chapter 7, Chapter 8, Chapter 9, Chapter 10, Chapter 11, Chapter 12, Chapter 17, Chapter 18, Chapter 21, Chapter 24, Chapter 33, Chapter 45, Chapter 53, Chapter 54, Chapter 55, Chapter 59,

Cluster 2 words: b'wickham', b'gardiner', b'lydia', b'letter', b'mr.', b'mrs.', b'aunt', b'married', b'love', b'shall',

Cluster 2 titles: Chapter 25, Chapter 26, Chapter 27, Chapter 34, Chapter 35, Chapter 36, Chapter 39, Cha

## k = 4 output

In [143]:
from sklearn.cluster import KMeans

num_clusters = 4

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

from sklearn.externals import joblib

#uncomment the below to save your model 
#since I've already run my model I am loading from the pickle

joblib.dump(km,  'doc_cluster.pkl')

km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

ppbook = {'title': titles, 'synopsis': synopses, 'cluster': clusters}

frame = pd.DataFrame(ppbook, index = [clusters] , columns = ['title', 'cluster'])
frame['cluster'].value_counts() #number of films per cluster (clusters from 0 to 4)

from __future__ import print_function

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :10]: #replace 6 with n words per cluster
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print() #add whitespace
    print() #add whitespace
    
    print("Cluster %d titles:" % i, end='')
    for title in frame.ix[i]['title'].values.tolist():
        print(' %s,' % title, end='')
    print() #add whitespace
    print() #add whitespace
    
print()
print()

CPU times: user 475 ms, sys: 5.02 ms, total: 480 ms
Wall time: 484 ms
Top terms per cluster:

Cluster 0 words: b'lydia', b'wickham', b'gardiner', b'forster', b'married', b'kitty', b'letter', b'uncle', b'lydia', b'mr.',

Cluster 0 titles: Chapter 27, Chapter 39, Chapter 41, Chapter 42, Chapter 46, Chapter 47, Chapter 48, Chapter 49, Chapter 50, Chapter 51, Chapter 52,

Cluster 1 words: b'bingley', b'mr.', b'miss', b'love', b'letter', b'walking', b'happy', b'wickham', b'netherfield', b'gardiner',

Cluster 1 titles: Chapter 7, Chapter 8, Chapter 10, Chapter 11, Chapter 12, Chapter 21, Chapter 24, Chapter 25, Chapter 26, Chapter 32, Chapter 33, Chapter 34, Chapter 35, Chapter 36, Chapter 40, Chapter 43, Chapter 44, Chapter 45, Chapter 53, Chapter 54, Chapter 55, Chapter 58, Chapter 59, Chapter 60,

Cluster 2 words: b'collins', b'mr.', b'lady', b'catherine', b'charlotte', b'ladyship', b'daughters', b'rosings', b'cousin', b'sir',

Cluster 2 titles: Chapter 13, Chapter 14, Chapter 15, Chapter

### k = 10 output

In [189]:
from sklearn.cluster import KMeans

num_clusters = 10

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

from sklearn.externals import joblib

#uncomment the below to save your model 
#since I've already run my model I am loading from the pickle

joblib.dump(km,  'doc_cluster.pkl')

km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

ppbook = {'title': titles, 'synopsis': synopses, 'cluster': clusters}

frame = pd.DataFrame(ppbook, index = [clusters] , columns = ['title', 'cluster'])
frame['cluster'].value_counts() #number of films per cluster

from __future__ import print_function

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :5]: #replace 5 with n words per cluster -- For Word2Vec used 10 to find the adj and verb
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print() #add whitespace
    print() #add whitespace
    
    print("Cluster %d titles:" % i, end='')
    for title in frame.ix[i]['title'].values.tolist():
        print(' %s,' % title, end='')
    print() #add whitespace
    print() #add whitespace
    
print()
print()

CPU times: user 739 ms, sys: 15.1 ms, total: 754 ms
Wall time: 440 ms
Top terms per cluster:

Cluster 0 words: b'mr.', b'collins', b'bingley', b'suppose', b'colonel',

Cluster 0 titles: Chapter 24, Chapter 32, Chapter 33,

Cluster 1 words: b'bingley', b'mr.', b'mrs.', b'mother', b'mr.',

Cluster 1 titles: Chapter 1, Chapter 2, Chapter 7, Chapter 9, Chapter 12, Chapter 21, Chapter 53, Chapter 54, Chapter 55,

Cluster 2 words: b'collins', b'charlotte', b'mr.', b'lady', b'catherine',

Cluster 2 titles: Chapter 28, Chapter 29, Chapter 30, Chapter 38,

Cluster 3 words: b'dance', b'wickham', b'mr.', b'bingley', b'mr.',

Cluster 3 titles: Chapter 3, Chapter 15, Chapter 16, Chapter 17, Chapter 18,

Cluster 4 words: b'gardiner', b'mrs.', b'bingley', b'pemberley', b'miss',

Cluster 4 titles: Chapter 25, Chapter 42, Chapter 43, Chapter 44, Chapter 45,

Cluster 5 words: b'lydia', b'wickham', b'gardiner', b'letter', b'kitty',

Cluster 5 titles: Chapter 26, Chapter 27, Chapter 39, Chapter 41, Chapte

# Part 2: Word2Vec


### For Word2Vec, I had a hard time finding adjectives and verbs from my initial hypothesis clustering (k=3) -- so I used k=10 clustering printing 10 words for each cluster to determine some of these words (double checked with Andrea - removed the words down to 5 in order to save space on the notebook and make it not appeared cluttered). 


For the nouns, I looked at “letter,” “dance,” “ball,” “miss,” and “daughter.” What I found particularly interesting is when i used ‘dance (minus ‘move’)’ vs ‘ball (minus ‘round’)’ - what I found interesting is that word2vec had similar results (albeit ‘dance’ was more accurate, capturing words like ‘party’), but word net was entirely off, citing words like ‘musket ball.’ I also found that some of the words could easily be substituted from word2vec, like ‘ladies’ being substituted for ‘miss,’ while other words could not be subbed into Pride and Prejudice, like the word2vec ‘memo’ could never substitute for ‘letter’!

For the verbs, I looked at ‘married,’ ‘dance,’ ‘read,’ ‘love,’ and ‘laugh.’ These were also very close, and could be substituted in a way that made sense in the text, but again word2vec overall did better at capturing similar words than word net (see ‘joke’ being substituted for ‘laugh’ - that would not work in PnP).

Lastly, for the adjectives, I looked at ‘happy,’ ‘mean,’ ‘dear,’ ‘young,’ and ‘young (minus teenager)’. I had a harder time finding adjectives in my text from the clustering, even with k=10 and 10 words output, which is why I tried to compare young and young (minus teenager). Word2Vec for ‘young’ returns nouns, while I wanted adjectives, which is why i removed ‘teenager,’ and although it started returning adjectives (‘youngish,’ ‘talented,’ etc) it still could not be substituted. Wordnet only returned nouns for all of the adjectives except ‘happy.’ Interestingly enough, word2vec and word net returned ‘mean’ as like ‘means to an end’ - not an adjective, even after I set it on negative=imply (then it was more words like ‘heck’ and ‘gosh’).

Overall, I think Jane Austen has a certain way of speaking that is not easily substitutable, but the best way to match her words (to get similar meanings) would be looking at nouns in word2vec.

----------------------------------

*Instruction: Experiment with Word2Vec to find related terms for terms in your collection.  I recommend using the large pre-trained collection that is in the notebook we discussed in class.   You can do this one of two ways.  Either follow the instructions shown below, or come up with your own way to explore with it.*
*(a) Select five nouns of interest from your collection, and compare what WordNet finds as the first 3 synsets to what Word2Vec finds as the top 5 rated similar nouns (using the most_similar() function).  State results are better for your collection in each case?  (you may use negative evidence if you like, by providing positive and negative example words).*
*(b) Do the same for 5 adjectives.*
*(c) Do the same for 5 verbs.*

In [128]:
import nltk
import numpy as np
import gensim
from gensim.models import Word2Vec
from nltk.data import find
#from nltk.download import word2vec_sample
# word2vec_sampler=nltk.data.load('models/word2vec_sample/pruned.word2vec.txt')
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.Word2Vec.load_word2vec_format(word2vec_sample, binary=False)

300

### Nouns

In [198]:
print("\n Letter:")
print('\t Word2Vec:') 
print(model.most_similar(positive=['letter'], topn = 5))
print('\t Wordnet:') 
print(wn.synsets('letter')[:3])

print("\n \n Dance:")
print('\t Word2Vec:') 
print(model.most_similar(positive=['dance', 'party'], negative=['move'], topn = 5))
print('\t Wordnet:') 
print(wn.synsets('dance')[:3])

print("\n \n Ball:")
print('\t Word2Vec:') 
print(model.most_similar(positive=['ball', 'dance'], negative=['round'], topn = 5))
print('\t Wordnet:') 
print(wn.synsets('ball')[:3])

print("\n \n Miss:")
print('\t Word2Vec:') 
print(model.most_similar(positive=['miss', 'lady'], negative=['skip'], topn = 5))
print('\t Wordnet:') 
print(wn.synsets('miss')[:3])


print("\n \n Daughter:")
print('\t Word2Vec:') 
print(model.most_similar(positive=['daughter'], topn = 5))
print('\t Wordnet:') 
print(wn.synsets('daughter')[:3])


 Letter:
	 Word2Vec:
[('memo', 0.7355137467384338), ('letters', 0.7233610153198242), ('missive', 0.6917874813079834), ('memorandum', 0.6289511919021606), ('statement', 0.6102292537689209)]
	 Wordnet:
[Synset('letter.n.01'), Synset('letter.n.02'), Synset('letter.n.03')]

 
 Dance:
	 Word2Vec:
[('dances', 0.5201530456542969), ('dancing', 0.5081790685653687), ('Party', 0.4721127450466156), ('parties', 0.4581678807735443), ('Dance', 0.4439452886581421)]
	 Wordnet:
[Synset('dance.n.01'), Synset('dance.n.02'), Synset('dancing.n.01')]

 
 Ball:
	 Word2Vec:
[('dancing', 0.561704158782959), ('dances', 0.5176332592964172), ('Dance', 0.49584853649139404), ('dancers', 0.47717174887657166), ('dancer', 0.47346895933151245)]
	 Wordnet:
[Synset('ball.n.01'), Synset('musket_ball.n.01'), Synset('ball.n.03')]

 
 Miss:
	 Word2Vec:
[('woman', 0.41856345534324646), ('gentleman', 0.39971935749053955), ('ladies', 0.38871461153030396), ('vivacious', 0.38015586137771606), ('hostess', 0.3671933710575104)]
	 Wo

### Verbs

In [197]:
print("\n Married:")
print('\t Word2Vec:')
print(model.most_similar(positive=['married'], topn = 5)) #I have 
print('\t Wordnet:') 
print(wn.synsets('married')[:3])


print("\n Dance:")
print('\t Word2Vec:')
print(model.most_similar(positive=['dance'], topn = 5))
print('\t Wordnet:') 
print(wn.synsets('dance')[:3])


print("\n Read:")
print('\t Word2Vec:')
print(model.most_similar(positive=['read'], topn = 5))
print('\t Wordnet:') 
print(wn.synsets('read')[:3])


print("\n Love:")
print('\t Word2Vec:')
print(model.most_similar(positive=['love'], topn = 5))
print('\t Wordnet:') 
print(wn.synsets('love')[:3])


print("\n Laugh:")
print('\t Word2Vec:')
print(model.most_similar(positive=['laugh'], topn = 5))
print('\t Wordnet:') 
print(wn.synsets('laugh')[:3])


 Married:
	 Word2Vec:
[('divorced', 0.8280808925628662), ('wed', 0.7852946519851685), ('marry', 0.7635562419891357), ('remarried', 0.7542672753334045), ('marrying', 0.7138937711715698)]
	 Wordnet:
[Synset('married.n.01'), Synset('marry.v.01'), Synset('marry.v.02')]

 Dance:
	 Word2Vec:
[('dancing', 0.8380804061889648), ('dances', 0.8213123083114624), ('dancers', 0.7513905763626099), ('Dance', 0.7395395040512085), ('dancer', 0.6916880011558533)]
	 Wordnet:
[Synset('dance.n.01'), Synset('dance.n.02'), Synset('dancing.n.01')]

 Read:
	 Word2Vec:
[('reading', 0.698559045791626), ('reread', 0.6906844973564148), ('reads', 0.662841260433197), ('write', 0.6306530237197876), ('written', 0.5812059640884399)]
	 Wordnet:
[Synset('read.n.01'), Synset('read.v.01'), Synset('read.v.02')]

 Love:
	 Word2Vec:
[('loved', 0.6907792091369629), ('adore', 0.681687593460083), ('loves', 0.6618633270263672), ('passion', 0.6100708246231079), ('hate', 0.6003957986831665)]
	 Wordnet:
[Synset('love.n.01'), Synset(

### Adjective

In [200]:
print("\n Happy:")
print('\t Word2Vec:')
print(model.most_similar(positive=['happy'], topn = 5))
print('\t Wordnet:') 
print(wn.synsets('happy')[:3])


print("\n Mean:")
print('\t Word2Vec:')
print(model.most_similar(positive=['mean'], topn = 5))
print('\t Wordnet:') 
print(wn.synsets('mean')[:3])


print("\n Dear:")
print('\t Word2Vec:')
print(model.most_similar(positive=['dear'], topn = 5))
print('\t Wordnet:') 
print(wn.synsets('dear')[:3])


print("\n Young:")
print('\t Word2Vec:')
print(model.most_similar(positive=['young'], topn = 5))
print('\t Wordnet:') 
print(wn.synsets('young')[:3])


print("\n Young (negative= 'teenager'):")
print('\t Word2Vec:')
print(model.most_similar(positive=['young'], negative=['teenager'], topn = 5))
print('\t Wordnet:') 
print(wn.synsets('young')[:3])


 Happy:
	 Word2Vec:
[('glad', 0.7408891320228577), ('pleased', 0.6632169485092163), ('ecstatic', 0.6626911759376526), ('thrilled', 0.6514050364494324), ('satisfied', 0.6437950134277344)]
	 Wordnet:
[Synset('happy.a.01'), Synset('felicitous.s.02'), Synset('glad.s.02')]

 Mean:
	 Word2Vec:
[('means', 0.6425307989120483), ('imply', 0.620326042175293), ('equate', 0.5891611576080322), ('necessarily', 0.572253406047821), ('necessitate', 0.569510817527771)]
	 Wordnet:
[Synset('mean.n.01'), Synset('mean.v.01'), Synset('entail.v.01')]

 Dear:
	 Word2Vec:
[('dearest', 0.6255167126655579), ('dearly', 0.563401997089386), ('beloved', 0.47946229577064514), ('cherished', 0.46813926100730896), ('cherish', 0.45106256008148193)]
	 Wordnet:
[Synset('beloved.n.01'), Synset('lamb.n.04'), Synset('beloved.s.01')]

 Young:
	 Word2Vec:
[('teenage', 0.6431924104690552), ('younger', 0.6350939869880676), ('youth', 0.5715783834457397), ('youngsters', 0.5662700533866882), ('teenagers', 0.5633072257041931)]
	 Wordn